# Регрессионный анализ

## Библиотеки

In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib.pylab as plt
import pandas as pd
import seaborn as sns
from statsmodels.regression.linear_model import OLS
from statsmodels.stats.diagnostic import het_breuschpagan
import statsmodels.api as sm

from patsy.contrasts import Treatment
from patsy.contrasts import Sum

## Warmup

### Данные

In [ ]:
rs = np.random.RandomState(42)
X = rs.randn(1000, 2)
w = np.array([1, 1])
y = np.dot(X, w) + rs.randn(1000) * 0.1
noise = rs.randn(1000)

In [ ]:
plt.scatter(noise, y)
plt.show()

In [ ]:
plt.scatter(X[:, 0], y)
plt.show()

In [ ]:
plt.scatter(X[:, 1], y)
plt.show()

In [ ]:
plt.scatter(np.sum(X, 1), y)
plt.show()

### Линейная регрессия

#### Модель без шума из одного признака

##### Инициализация через аргументы

In [ ]:
X.shape, y.shape

In [ ]:
simple_model = OLS(y, X[:, 0]).fit()
simple_model.summary()

##### Инициализация через формулы

In [ ]:
simple_model = OLS.from_formula('y ~ x1 - 1', {'y' : y, 'x1' : X[:, 0]}).fit()
simple_model.summary()

#### Модель шума

In [ ]:
noise_model = OLS(y, noise).fit()
noise_model.summary()

#### Модель без шума из двух признаков

In [ ]:
complex_model = OLS.from_formula('y ~ x1 + x2 - 1', {'y' :  y, 'x1' : X[:, 0], 'x2' : X[:, 1]}).fit()
complex_model.summary()

#### Сравнение вложенных моделей
Каждый тест возвращает:

- значение статистики (так как, мы исследуем важность только одного признака, F-критерий эквивалентен критерию Стьюдента для двусторонней альтернативы)
- p-value
- разница в количестве степеней свободы между моделями

##### Критерий Фишера

In [ ]:
complex_model.compare_f_test(simple_model)

##### Критерий отношения правдоподобия ($H_0$: равенство правдоподобий двух моделей)

In [ ]:
complex_model.compare_lr_test(simple_model)

##### Критерий Бройша-Пагана (множителей Лагранжа)

In [ ]:
complex_model.compare_lm_test(simple_model)

Нулевая гипотеза об отсутствии влияния второго признака на целевую переменную отвергается

#### Добавим шум

In [ ]:
complex_model2 = OLS.from_formula('y ~ x1 + x2 + noise - 1',
                                  {
                                      'y' : y,
                                      'noise' : noise,
                                      'x1' : X[:, 0],
                                      'x2' : X[:, 1]
                                  }).fit()
complex_model2.summary()

In [ ]:
complex_model2.compare_lm_test(complex_model)

In [ ]:
complex_model2.compare_lm_test(noise_model)

In [ ]:
complex_model2.summary()

In [ ]:
complex_model.summary()

In [ ]:
noise_model.summary()

### Выбор моделей: случай невложенных моделей

In [ ]:
rs = np.random.RandomState(42)
X = rs.randn(100)
y = X**2 + rs.randn(100) * 0.1

In [ ]:
plt.scatter(X, y)

plt.show()

In [ ]:
plt.scatter(X ** 2, y)
plt.show()

#### Линейная модель

In [ ]:
model1 = OLS(y, X).fit()
model1.summary()

#### Квадратичная модель

In [ ]:
model2 = OLS(y, X ** 2).fit()
model2.summary()

#### Предсказания моделей

In [ ]:
y_predicted1 = model1.predict(X)
y_predicted2 = model2.predict(X ** 2)

In [ ]:
plt.scatter(y_predicted1, y - y_predicted1)
plt.xlabel('$\hat y_1$')
_ = plt.ylabel('$\epsilon$')
plt.show()

In [ ]:
plt.scatter(y_predicted2, y)
plt.xlabel('$\hat y_2$')
plt.ylabel('y')
plt.show()

#### Критерий Давидсона-Маккиннона

In [ ]:
data = {'y' : y, 'y1' : y_predicted1, 'y2': y_predicted2, 'X' : X, 'X2' : X ** 2}
model1_with_y_2 = OLS.from_formula('y ~ y2 + X', data=data).fit()
model2_with_y_1 = OLS.from_formula('y ~ y1 + X2', data=data).fit()

In [ ]:
model1_with_y_2.summary()

In [ ]:
model2_with_y_1.summary()

### Кодирование категориальных переменных

In [ ]:
rs = np.random.RandomState(42)

data = [1] * 10 + [2] * 7 + [3] * 5
rs.shuffle(data)
data = np.array(data)
data

#### dummy

In [ ]:
levels = [1, 2, 3]
contrast = Treatment().code_without_intercept(levels)
print(contrast.matrix)

In [ ]:
contrast.matrix[data - 1]

#### deviation

In [ ]:
contrast = Sum().code_without_intercept(levels)
print(contrast.matrix)

In [ ]:
contrast.matrix[data - 1]

### Метод Бокса-Кокса

In [ ]:
rs = np.random.RandomState(42)
y = rs.randn(100)
print(np.array(y > 1).sum())

#### Изменим хвост нормального распределения

In [ ]:
_ = plt.hist(y)
plt.show()

In [ ]:
tails = (y) > 1
y[tails] *= 1.45

y = y - np.min(y) + 1
_ = plt.hist(y)
plt.show()

In [ ]:
def add_titlebox(ax, text):
    ax.text(.55, .8, text,
            horizontalalignment='center',
            transform=ax.transAxes,
            bbox=dict(facecolor='white', alpha=0.6),
            fontsize=12.5)
    return ax

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(9, 7))
lambdas = [-5, -2, -1, -0.5, 0, 0.5, 1, 2, 5]
for i, l in enumerate(lambdas):
    axes[i // 3, i % 3].hist(st.boxcox(y, l))
    add_titlebox(axes[i // 3, i % 3], 'lambda = ' + str(l))

In [ ]:
for l in np.arange(-2.0,  2.0, 0.1):    
    print(l, st.shapiro(st.boxcox(y, l)))

In [ ]:
bc, l = st.boxcox(y)
print(f'optimal lambda = {l}')
print('Box-Cox transformed array:')
print(bc)

### Гетероскедастичность

In [ ]:
rs = np.random.RandomState(42)
X = rs.randn(100)
X.sort()
error = rs.randn(100) * 0.1 * np.arange(100)
y = X + error
y = y - np.min(y) + 1
plt.scatter(X, y)
plt.xlabel('X')
plt.ylabel('y')
plt.show()

In [ ]:
model = OLS(y, X).fit()
predicted = model.predict(X)
plt.scatter(predicted, y - predicted)
plt.xlabel('$\hat y$')
_ = plt.ylabel('$\epsilon$')
plt.show()

### Преобразование Бокса-Кокса вручную

In [ ]:
def W(y, lam):
    return np.log(y) if lam == 0 else (y**lam - 1) / lam

In [ ]:
for l in [-2, -1, -0.5, 0, 0.5, 1, 2]:
    print(np.sum(W(y, l) - st.boxcox(y, l)))

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(9, 7))
fig.suptitle('Residual from predicted y')

model = OLS(y, X).fit()
predicted = model.predict(X)
axes[0, 0].scatter(predicted, y - predicted)
axes[0, 0].set_ylim((-2, 50))
add_titlebox(axes[0, 0], 'original')
print(max(y))
    
lams = [-15, -10, -5, -1, -0.5, 0, 0.5, 1]
for i, l in enumerate(lams):
    i += 1
    model = OLS(W(y, l), X).fit()
    predicted = model.predict(X)
    axes[i // 3, i % 3].scatter(predicted, W(y, l) - predicted)
    add_titlebox(axes[i // 3, i % 3], 'lambda = ' + str(l))
    axes[i // 3, i % 3].set_ylim((0, 50))

In [ ]:
lams = np.arange(-5, 5, 0.1)
r = []
for l in lams:
    model = OLS(W(y, l), X).fit()
    r.append(np.log(model.mse_resid))
plt.plot(lams, r)
plt.xlabel('$\lambda$')
plt.ylabel('mse')
plt.title('Mse from lambda ')
lams[np.argmin(r)]

#### Before

In [ ]:
model = OLS(y, X).fit()
print(model.mse_resid)

In [ ]:
model.summary()

#### After

In [ ]:
bc, l = st.boxcox(y)
model = OLS(bc, X).fit()
print(l, model.mse_resid)

In [ ]:
model.summary()

In [ ]:
model = OLS(W(y, -5), X).fit()
print(model.mse_resid)

In [ ]:
model.summary()

In [ ]:
plt.scatter(X, y)
plt.show()

In [ ]:
plt.scatter(st.boxcox(y)[0], X)
plt.show()

In [ ]:
plt.scatter(W(y, -5), X)
plt.show()

## Привлекательность и уровень заработной платы

По 1260 опрошенным имеются следующие данные:

* `wage` - заработная плата за час работы, $;
* `exper` - опыт работы, лет;
* `educ` - образование, лет;
* `looks` - внешняя привлекательность, в баллах от 1 до 5;
* бинарные признаки: 
    * `female` - пол, 
    * `married` - семейное положение, 
    * `goodhlth` - состояние здоровья (хорошее/плохое), 
    * `union` - членство в профсоюзе, 
    * `black` - цвет кожи (белый/чёрный), 
    * `service` - занятость в сфере обслуживания (да/нет).

Требуется оценить влияние внешней привлекательности на уровень заработка с учётом всех остальных факторов.



In [ ]:
data = pd.read_csv('data/beauty.csv', delimiter=';')
data.head()

In [ ]:
sns.pairplot(data[['wage ', 'exper', 'educ', 'looks']])
plt.show()

### Предобработка

#### Распределение оценок привлекательности: 

In [ ]:
_ = plt.hist(data['looks'])
plt.show()

#### В группах looks=1 и looks=5 слишком мало наблюдений. Превратим признак looks в категориальный и закодируем с помощью фиктивных переменных:


In [ ]:
data['below_avg'] = data['looks'].apply(lambda x: 1 if x < 3 else 0)
data['above_avg'] = data['looks'].apply(lambda x: 1 if x > 3 else 0)
looks = data.looks.copy()
data.drop(columns='looks', inplace=True)

data.head()

#### Распределение значений отклика:

In [ ]:
plt.figure(figsize=(16, 7))
plt.subplot(121)
data['wage '].plot.hist()
plt.xlabel('wage', fontsize=14)

plt.subplot(122)
np.log(data['wage ']).plot.hist()
plt.xlabel('Log wage', fontsize=14)
plt.show()

#### Уберем выброс

In [ ]:
sns.boxplot(data=data['wage '])
plt.show()

In [ ]:
data = data[data['wage '] < 70]

### Модель 0: Линейная регрессия зарплаты от всех признаков

In [ ]:
feat_names = [f for f in data.columns if f not in ['wage ']]
features = data[feat_names]
formula0 = ' '.join(['wage ~', 
                    ' + '.join([f for f in feat_names])])
formula0

In [ ]:
data = data.rename(columns=dict(
    zip(data.columns, list(
        map(lambda x: x if x != 'wage ' else 'wage', data.columns)))))

In [ ]:
model0 = sm.OLS.from_formula(formula0, data).fit()
model0.summary()

#### Проанализируем ошибки

In [ ]:
plt.figure(figsize=(16, 7))
plt.subplot(121)
st.probplot(model0.resid, plot=plt)

plt.subplot(122)
model0.resid.plot.hist()
plt.xlabel('Residuals', fontsize=14)
plt.show()

##### Бокса-Кокс

In [ ]:
max(data['wage']) / min(data['wage'])

$\frac{max y}{min y} \approx 40>10$, можем воспользоваться боксом коксом.

Возьмём $λ=0$, то есть, будем строить регрессию логарифма отклика.

In [ ]:
log_wage = st.boxcox(data['wage'], 0)
old_columns = list(data.columns)
data1 = data.copy()
data1['log_wage'] = log_wage

In [ ]:
plt.hist(log_wage)
plt.xlabel('Log wage', fontsize=14)
plt.show()

### Модель 1: зависимости логарифма заработка по всем признакам

In [ ]:
features = data1[feat_names]
formula1 = ' '.join(['log_wage ~', 
                    ' + '.join([f for f in feat_names])])
print(formula1)

In [ ]:
model1 = sm.OLS.from_formula(formula1, data1).fit()
model1.summary()

In [ ]:
plt.figure(figsize=(16, 7))
plt.subplot(121)
st.probplot(model1.resid, plot=plt)

plt.subplot(122)
model1.resid.plot.hist()
_ = plt.xlabel('Residuals', fontsize=14)
plt.show()

#### Проверка гипотез по остаткам

In [ ]:
print(f'Нормальность ошибок: {st.shapiro(model1.resid)}')
print(f'Равенство распределений: {st.wilcoxon(model1.resid)}')
print(f'Гомоскедастичность: {het_breuschpagan(model1.resid, features)}')

#### Рассмотрим зависимость от опыта и образования (самое очевидное)

In [ ]:
_ = sns.pairplot(pd.DataFrame({
    'exper': features['exper'], 
    'educ': features['educ'], 
    'residuals': model1.resid
}))

In [ ]:
sns.residplot(x=features['exper'], 
              y=model1.resid,
              lowess=True, 
              scatter_kws={'alpha': 0.5}, 
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
_ = plt.ylabel('residuals')

### Модель 2: добавим в модель 1 квадрат опыта работы

In [ ]:
exper2 = features['exper']**2
old_columns = list(data1.columns)
data2 = data1.copy()
data2['exper2'] = exper2
data2.head()

In [ ]:
feat_names += ['exper2']

In [ ]:
features = data2[feat_names]
formula2 = ' '.join(['log_wage ~', 
                    ' + '.join([f for f in feat_names])])
print(formula2)

In [ ]:
model2 = sm.OLS.from_formula(formula2, data2).fit()
model2.summary()

In [ ]:
gridsize = (2, 2)
fig = plt.figure(figsize=(16, 14))
ax1 = plt.subplot2grid(gridsize, (0, 0))
ax2 = plt.subplot2grid(gridsize, (0, 1))
ax3 = plt.subplot2grid(gridsize, (1, 0))
ax4 = plt.subplot2grid(gridsize, (1, 1))
st.probplot(model2.resid, plot=ax1)

model2.resid.plot.hist(ax=ax2)
ax2.set_xlabel('Residuals')

sns.residplot(x=features['exper'], 
              y=model2.resid,
              lowess=True, 
              scatter_kws={'alpha': 0.5}, 
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
              ax=ax3)
ax3.set_ylabel('residuals')
sns.residplot(x=features['exper2'], 
              y=model2.resid,
              lowess=True, 
              scatter_kws={'alpha': 0.5}, 
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8},
              ax=ax4)
_ = ax4.set_ylabel('residuals')

plt.show()

#### Проверка гипотез по остаткам

In [ ]:
print(f'Нормальность ошибок: {st.shapiro(model2.resid)}')
print(f'Равенство распределений: {st.wilcoxon(model2.resid)}')
print(f'Гомоскедастичность: {het_breuschpagan(model2.resid, features)}')

Незначимые признаки: `goodhlth`, `black`, `married`, `above_avg`. Прежде, чем удалять лишние признаки, проверим, не входят ли они в значимые попарные взаимодействия:

In [ ]:
for f1 in range(len(feat_names)):
    for f2 in range(f1 + 1, len(feat_names)):
        formula.append(feat_names[f1] + ':' + feat_names[f2])
formula = ' + '.join(formula)
formula

In [ ]:
model_all = sm.OLS.from_formula(formula, data2).fit()

anova = sm.stats.anova_lm(model_all)
bad_names = []
for id, p in enumerate(anova['PR(>F)']):
    if p < 0.05:
        print(anova.iloc[id].name, p)
    else:
        bad_names.append(anova.iloc[id].name)

In [ ]:
_ = sns.pairplot(pd.DataFrame({
    'exper': features['exper'], 
    'exper2': features['exper2'],
    'educ': features['educ'], 
    'residuals': model_all.resid
}))

In [ ]:
plt.figure(figsize=(16, 7))
plt.subplot(121)
_ = st.probplot(model_all.resid, plot=plt)

plt.subplot(122)
_ = st.probplot(model2.resid, plot=plt)
plt.show()

### Модель 3: удалим из модели 2 незначимые признаки (`goodhlth`, `black`, `married`) и добавим межфакторное взаимодействие пола и опыта работы, при этом оставим `above_avg`, так как мы хотим оценить влияние внешней привлекательности на зарплату

In [ ]:
bad_feats = ['goodhlth', 'black', 'married']
formula3 = ' '.join(['log_wage ~', 
                    ' + '.join([f for f in features if f not in bad_feats] + ['exper:female'])])
print(formula3)

In [ ]:
model3 = sm.OLS.from_formula(formula3, data=data2).fit()
model3.summary()

#### Проверка гипотез по остаткам

In [ ]:
print('Model 3')
print(st.shapiro(model3.resid))
print(st.wilcoxon(model3.resid))
print('Model 2')
print(st.shapiro(model2.resid))
print(st.wilcoxon(model2.resid))

In [ ]:
_ = sns.pairplot(pd.DataFrame({
    'exper': features['exper'], 
    'educ': features['educ'], 
    'residuals': model3.resid
}))

In [ ]:
st.probplot(model3.resid, plot=plt)
plt.show()

In [ ]:
model2.compare_lr_test(model3), model2.compare_f_test(model3)

### Модель 4: попробуем оставить в модели 2 цвет кожи и семейное положение, чтобы добавить их взаимодействия с полом. Как и в модели 3, добавим взаимодействие пола с опытом работы, а состояние здоровья удалим.

In [ ]:
bad_feats = ['goodhlth']
add_feats = ['exper:female', 'black:female', 'married:female']
formula4 = ' '.join(['log_wage ~', 
                    ' + '.join([f for f in features if f not in bad_feats] + add_feats)])
print(formula4)

In [ ]:
model4 = sm.OLS.from_formula(formula4, data=data2).fit()
model4.summary()

#### Проверка гипотез по остаткам

In [ ]:
print(st.shapiro(model4.resid))
print(st.wilcoxon(model4.resid))

In [ ]:
_ = sns.pairplot(pd.DataFrame({
    'exper': features['exper'], 
    'looks': looks,
    'residuals': model4.resid
}))
plt.show()

In [ ]:
plt.figure(figsize=(16, 7))
plt.subplot(121)
_ = st.probplot(model4.resid, plot=plt)

plt.subplot(122)
_ = st.probplot(model3.resid, plot=plt)
plt.show()

### Модель 5: в предыдущей модели семейное положение незначимо; посмотрим, можно ли удалить его

In [ ]:
bad_feats = ['goodhlth', 'married']
add_feats = ['exper:female', 'black:female', 'married:female']
formula5 = ' '.join(['log_wage ~', 
                    ' + '.join([f for f in features if f not in bad_feats] + add_feats)])
print(formula5)

In [ ]:
model5 = sm.OLS.from_formula(formula5, data=data2).fit()
model5.summary()

#### Проверка гипотез по остаткам

In [ ]:
print('Model 5')
print(st.shapiro(model5.resid))
print(st.wilcoxon(model5.resid))

print('Model 4')
print(st.shapiro(model4.resid))
print(st.wilcoxon(model4.resid))

In [ ]:
plt.figure(figsize=(16, 7))
plt.subplot(121)
_ = st.probplot(model4.resid, plot=plt)

plt.subplot(122)
_ = st.probplot(model5.resid, plot=plt)
plt.show()

##### Модель 5 по остаткам немного хуже

### Расстояние Кука
Посмотрим на влиятельные наблюдения: 
    

In [ ]:
cook = model4.get_influence().summary_frame().loc[:,'cooks_d']

In [ ]:
plt.scatter(data2['log_wage'], cook)
plt.xlim((0,4))
plt.ylim((-0.01, 0.05))
plt.show()

In [ ]:
data3 = data2[cook < 0.015]
data3.shape

In [ ]:
model6 = sm.OLS.from_formula(formula4, data=data3).fit()
model6.summary()

#### Проверка нормальности остатков

In [ ]:
print(st.shapiro(model6.resid))
print(st.shapiro(model4.resid))

In [ ]:
plt.figure(figsize=(16, 7))

plt.subplot(121)
plt.scatter(np.exp(model6.predict(data3)), data3['wage'])
plt.xlabel('wage', fontsize=14)
plt.ylabel('exp(predictions)', fontsize=14)

plt.subplot(122)
plt.scatter(model6.predict(data3), data3['log_wage'])
plt.xlabel('log wage', fontsize=14)
_ = plt.ylabel('predictions', fontsize=14)
plt.show()

### Итого

- Итоговая модель (№6) объясняет 43% вариации логарифма отклика
- Коэффициент при `below_avg`: -0.13 => человек, привлекательность которого ниже среднего, получает зарплату на 13% ниже, в среднем ($p=0.001$, 95%-й доверительный интервал: \[5,21\]%)
- Коэффициент при `above_avg`: -0.042 => человек, привлекательность которого выше среднего, примерно такую же, сколько и люди со средним уровнем привлекательности ($p =  0.884$, 95%-й доверительный интервал: \[-6, 6\]%), признак неинформативен